# Imports

In [2]:
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
np.random.seed(42)
import pandas as pd
import string
import re

import gensim
from collections import Counter
import pickle

import tensorflow as tf
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Dense, Dropout, Conv1D, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, LSTM,Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import CuDNNLSTM, CuDNNGRU
from keras.preprocessing import text
from keras.utils import pad_sequences

from keras.callbacks import Callback
from keras import optimizers
from keras.layers import Lambda

import warnings
warnings.filterwarnings('ignore')

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

import os
os.environ['OMP_NUM_THREADS'] = '4'

import gc
from keras import backend as K
from sklearn.model_selection import KFold

from unidecode import unidecode

import time

eng_stopwords = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Preprocessing
We have severals step in preprocessing:  
1) cleain_text: delete some useless charactes like digits and punctuation marks.  
2) revise_cuts: delete cuts like "you're" or "i'm" and so on.  
3) revise_star: due to stars often used for disguise bad words, we create set of the most popular bad words and check: "does the word in bad if delete all start?", if yes then delete stars.  
4) revise_triple_and_more_letters: delete char duplicates.  
5) revise_redundancy_words: delete redundancy of bad words.

In [4]:
replace_numbers = re.compile(r'\d+', re.IGNORECASE)

COMMENT_COL = 'comment_text'
ID_COL = 'id'
input_dir = './data/'
output_dir = './output/'
embeddings_dir = "./drive/MyDrive/embeddings/"

In [5]:
cuts = {
    "you're": ['you', 'are'],
    "i'm": ['i', 'am'],
    "he's": ['he', 'is'],
    "she's": ['she', 'is'],
    "it's": ['it', 'is'],
    "they're": ['they', 'are'],
    "can't": ['can', 'not'],
    "couldn't": ['could', 'not'],
    "don't": ['do', 'not'],
    "don;t": ['do', 'not'],
    "didn't": ['did', 'not'],
    "doesn't": ['does', 'not'],
    "isn't": ['is', 'not'],
    "wasn't": ['was', 'not'],
    "aren't": ['are', 'not'],
    "weren't": ['were', 'not'],
    "won't": ['will', 'not'],
    "wouldn't": ['would', 'not'],
    "hasn't": ['has', 'not'],
    "haven't": ['have', 'not'],
    "what's": ['what', 'is'],
    "that's": ['that', 'is'],
}
set_cuts = set(cuts.keys())

In [6]:
toxic_indicator_words = [
    'fuck', 'fucking', 'fucked', 'fuckin', 'fucka', 'fucker', 'fucks', 'fuckers',
    'fck', 'fcking', 'fcked', 'fckin', 'fcker', 'fcks',
    'fuk', 'fuking', 'fuked', 'fukin', 'fuker', 'fuks', 'fukers',
    'fk', 'fking', 'fked', 'fkin', 'fker', 'fks',
    'shit', 'shitty', 'shite',
    'stupid', 'stupids',
    'idiot', 'idiots',
    'suck', 'sucker', 'sucks', 'sucka', 'sucked', 'sucking',
    'ass', 'asses', 'asshole', 'assholes', 'ashole', 'asholes',
    'gay', 'gays',
    'niga', 'nigga', 'nigar', 'niggar', 'niger', 'nigger',
    'monster', 'monsters',
    'loser', 'losers',
    'nazi', 'nazis',
    'cock', 'cocks', 'cocker', 'cockers',
    'faggot', 'faggy',
]
toxic_indicator_words_sets = set(toxic_indicator_words)

In [7]:
redundancy_rightFormat = {
    'ckckck': 'cock',
    'fuckfuck': 'fuck',
    'lolol': 'lol',
    'lollol': 'lol',
    'pussyfuck':'fuck',
    'gaygay': 'gay',
    'haha': 'ha',
    'sucksuck': 'suck'}

redundancy = set(redundancy_rightFormat.keys())

In [8]:
class CommProcess(object):
    @staticmethod
    def clean_text(t):
        t = re.sub(r"[^A-Za-z0-9,!?*.;’´'\/]", " ", t)
        t = replace_numbers.sub(" ", t)
        t = t.lower()
        t = re.sub(r",", " ", t)
        t = re.sub(r"’", "'", t)
        t = re.sub(r"´", "'", t)
        t = re.sub(r"\.", " ", t)
        t = re.sub(r"!", " ! ", t)
        t = re.sub(r"\?", " ? ", t)
        t = re.sub(r"\/", " ", t)
        return t

    @staticmethod
    def revise_cuts(t):
        ret = []
        for word in t.split():
            if word in set_cuts:
                ret.append(cuts[word][0])
                ret.append(cuts[word][1])
            else:
                ret.append(word)
        ret = ' '.join(ret)
        ret = re.sub("'", " ", ret)
        ret = re.sub(r";", " ", ret)
        return ret

    @staticmethod
    def revise_star(t):
        ret = []
        for word in t.split():
            if ('*' in word) and (re.sub('\*', '', word) in toxic_indicator_words_sets):
                word = re.sub('\*', '', word)
            ret.append(word)
        ret = re.sub('\*', ' ', ' '.join(ret))
        return ret

    @staticmethod
    def revise_triple_and_more_letters(t):
        for letter in 'abcdefghijklmnopqrstuvwxyz':
            reg = letter + "{2,}"
            t = re.sub(reg, letter + letter, t)
        return t

    @staticmethod
    def revise_redundancy_words(t):
        ret = []
        for word in t.split(' '):
            for redu in redundancy:
                if redu in word:
                    word = redundancy_rightFormat[redu]
                    break
            ret.append(word)
        return ' '.join(ret)


def execute_comm_process(df):
    comm_process_pipeline = [
        CommProcess.clean_text,
        CommProcess.revise_cuts,
        CommProcess.revise_star,
        CommProcess.revise_triple_and_more_letters,
        CommProcess.revise_redundancy_words,
    ]
    for cp in comm_process_pipeline:
        df[COMMENT_COL] = df[COMMENT_COL].apply(cp)
    return df


## Read files
Read files and save 'processed' data train and test files using previous preprocessing.

In [11]:
df_train = pd.read_csv(input_dir + 'train.csv')
df_test = pd.read_csv(input_dir + 'test.csv')

In [12]:
df_train = execute_comm_process(df_train)
df_train.to_csv(input_dir + 'train_processed.csv', index=False)

df_test = execute_comm_process(df_test)
df_test.to_csv(input_dir+ 'test_processed.csv', index=False)


In [13]:
train = pd.read_csv(input_dir + 'train_processed.csv')
test = pd.read_csv(input_dir + 'test_processed.csv')

## Remove non-ascii characters

In [14]:
special_character_removal = re.compile(r'[^A-Za-z\.\-\?\!\,\#\@\% ]',re.IGNORECASE)
def clean_text(x):
    x_ascii = unidecode(x)
    x_clean = special_character_removal.sub('',x_ascii)
    return x_clean

train['comment_text'] = train['comment_text'].apply(lambda x: clean_text(str(x)))
test['comment_text'] = test['comment_text'].apply(lambda x: clean_text(str(x)))

X_train = train['comment_text'].fillna("something").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test['comment_text'].fillna("something").values

## Add features and scale data
New features will be used in the process of training and it can increase accuracy/score.

In [15]:
def add_features(df):
    df['comment_text'] = df['comment_text'].apply(str)
    df['total_length'] = df['comment_text'].apply(len)
    df['capitals'] = df['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    df['caps_vs_length'] = df.apply(lambda row: float(row['capitals'])/float(row['total_length']), axis=1)
    df['num_words'] = df.comment_text.str.count('\S+')
    df['num_unique_words'] = df['comment_text'].apply(lambda comment: len(set(w for w in comment.split())))
    df['words_vs_unique'] = df['num_unique_words'] / df['num_words']

    return df

# add features to dataset
train = add_features(train)
test = add_features(test)

# get features from dataset
features = train[['caps_vs_length', 'words_vs_unique']].fillna(0)
test_features = test[['caps_vs_length', 'words_vs_unique']].fillna(0)

# Initialize StandardScaler and scale features
ss = StandardScaler()
ss.fit(np.vstack((features, test_features)))
features = ss.transform(features)
test_features = ss.transform(test_features)

## Tokenize data

In [16]:
max_features = 300000
maxlen = 900

# Initialize tokenizer and tokenize dataset
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train_sequence = tokenizer.texts_to_sequences(X_train)
X_test_sequence = tokenizer.texts_to_sequences(X_test)

# Add pads into tokenized data to all sentences will be the same length
x_train = pad_sequences(X_train_sequence, maxlen=maxlen)
x_test = pad_sequences(X_test_sequence, maxlen=maxlen)
print(len(tokenizer.word_index))

283759


## Download Embeddings

In [17]:
# Load the FastText Web Crawl vectors
EMBEDDING_FILE_FASTTEXT = embeddings_dir + "fasttext-crawl-300d-2m/crawl-300d-2M.vec"
EMBEDDING_FILE_TWITTER = embeddings_dir + "glove-twitter-27b-200d-txt/glove.twitter.27B.200d.txt"
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index_ft = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE_FASTTEXT,encoding='utf-8'))
embeddings_index_tw = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE_TWITTER,encoding='utf-8'))

# Spelling correction
We used spelling correction which was used in the 2-nd assignment. It will used in the embedding step. We use spell correction in the situation when embedding can embed the word, so we assume that in the word can be some mistake or typo, we fix it using spell correction and try to use embedding again, but already on the correct word.

In the 2-nd assignment we used two boooks. Here we can use vocabulary from FASTTEXT.

In [18]:
spell_model = gensim.models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE_FASTTEXT)

In [19]:
words = spell_model.index_to_key

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

# Use fast text as vocabulary
def words(text): return re.findall(r'\w+', text.lower())

def P(word):
    """Probability of `word`."""
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word):
    """Most probable spelling correction for word."""
    return max(candidates(word), key=P)

def candidates(word):
    """Generate possible spelling corrections for word."""
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    """The subset of `words` that appear in the dictionary of WORDS."""
    return set(w for w in words if w in WORDS)

def edits1(word):
    """All edits that are one edit away from `word`."""
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    """All edits that are two edits away from `word`."""
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def singlify(word):
    return "".join([letter for i, letter in enumerate(word) if i == 0 or letter != word[i-1]])


# Embed words
Size of embeded vector is 501. And to get embeded vector we use tasttext and twitter embedding. First, we create embeded vector for word 'something' because in the preprocessing step we changed NaN to word 'something' so it better to have it. Then we generate embeded vectors for all tokenized words. We make vector where [:300,] are from fasttext and [300:500,] from twitter ebmbedding. Last 501-th number is show: does the word is the first word in the sentences or not(it's check the first letter of the word). If we can't get embedding vector from word, even with spell corecctions, then we make it like word "something" vector.

In [20]:
word_index = tokenizer.word_index
number_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((number_words,501))

something_tw = embeddings_index_tw.get("something")
something_ft = embeddings_index_ft.get("something")

something = np.zeros((501,))
something[:300,] = something_ft
something[300:500,] = something_tw
something[500,] = 0

def all_caps(word):
    return len(word) > 1 and word.isupper()

def embed_word(embedding_matrix, i, word):
    embedding_vector_ft = embeddings_index_ft.get(word)
    if embedding_vector_ft is not None:
        if all_caps(word):
            last_value = np.array([1])
        else:
            last_value = np.array([0])
        embedding_matrix[i,:300] = embedding_vector_ft
        embedding_matrix[i,500] = last_value
        embedding_vector_tw = embeddings_index_tw.get(word)
        if embedding_vector_tw is not None:
            embedding_matrix[i,300:500] = embedding_vector_tw

# Fasttext vector is used by itself if there is no glove vector but not the other way around.
for word, i in word_index.items():
    if i - 1 >= max_features: continue

    if embeddings_index_ft.get(word) is not None:
        embed_word(embedding_matrix,i-1,word)
    else:
        # change to > 20 for better score.
        if len(word) > 0:
            embedding_matrix[i-1] = something
        else:
            word2 = correction(word)
            if embeddings_index_ft.get(word2) is not None:
                embed_word(embedding_matrix,i-1,word2)
            else:
                word2 = correction(singlify(word))
                if embeddings_index_ft.get(word2) is not None:
                    embed_word(embedding_matrix,i-1,word2)
                else:
                    embedding_matrix[i-1] = something

# Evaluation
Roc Auc Score which is used in kaggle compition to calculate test score.

In [21]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data
        self.max_score = 0
        self.not_better_count = 0

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=1)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
            if (score > self.max_score):
                print("*** New High Score (previous: %.6f) \n" % self.max_score)
                model.save_weights("best_weights.h5")
                self.max_score=score
                self.not_better_count = 0
            else:
                self.not_better_count += 1
                if self.not_better_count > 3:
                    print("Epoch %05d: early stopping, high score = %.6f" % (epoch,self.max_score))
                    self.model.stop_training = True

# Model description
First layer: As We said latter: concatenated fasttext and glove twitter embeddings. Fasttext vector is used by itself if there is no glove vector but not the other way around. Words without word vectors are replaced with a word vector for a word "something". Also, We added additional value that was set to 1 if a word was written in all capital letters and 0 otherwise.

Second layer: SpatialDropout1D(0.5)

Third layer: Bidirectional CuDNNLSTM with a kernel size 40. We found out that LSTM as a first layer works better than GRU.

Fourth layer: Bidirectional CuDNNGRU with a kernel size 40.

Fifth layer: A concatenation of the last state, maximum pool, average pool and two features: "Unique words rate" and "Rate of all-caps words"

Sixth layer: output dense layer.

In [22]:
def get_model(features, clipvalue=1., num_filters=40, dropout=0.5, embed_size=501):
    features_input = Input(shape=(features.shape[1],))
    inp = Input(shape=(maxlen, ))

    # Layer 1: concatenated fasttext and glove twitter embeddings.
    x = Embedding(number_words, embed_size, weights=[embedding_matrix], trainable=False)(inp)

    # Layer 2: SpatialDropout1D(0.5)
    x = SpatialDropout1D(dropout)(x)

    # Layer 3: Bidirectional CuDNNLSTM
    x = Bidirectional(LSTM(num_filters, return_sequences=True))(x)

    # Layer 4: Bidirectional CuDNNGRU
    x, x_h, x_c = Bidirectional(GRU(num_filters, return_sequences=True, return_state = True))(x)

    # Layer 5: A concatenation of the last state, maximum pool, average pool and
    # two features: "Unique words rate" and "Rate of all-caps words"
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)

    x = concatenate([avg_pool, x_h, max_pool,features_input])

    # Layer 6: output dense layer.
    outp = Dense(6, activation="sigmoid")(x)

    model = Model(inputs=[inp,features_input], outputs=outp)
    adam = optimizers.adam(clipvalue=clipvalue)
    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
    return model

# Train model

In [ ]:
model = get_model(features)

batch_size = 32

epochs = 10
gc.collect()
K.clear_session()

num_folds = 10 

predict = np.zeros((test.shape[0],6))

scores = []
oof_predict = np.zeros((train.shape[0],6))

kf = KFold(n_splits=num_folds, shuffle=True, random_state=239)

for train_index, test_index in KFold(n_splits=10, shuffle=True, random_state=239).split(x_train):
    x_train_new = x_train[train_index]
    features_train_new = features[train_index]
    y_train_new = y_train[train_index]

    x_test_new = x_train[test_index]
    features_test_new = features[test_index]
    y_test_new = y_train[test_index]
    
    for _train_index, val_index in kf.split(x_train_new):
        kfold_X_train = x_train_new[_train_index]
        kfold_X_features = features_train_new[_train_index]
        kfold_y_train = y_train_new[_train_index]

        kfold_X_val = x_train_new[val_index]
        kfold_X_val_features = features_train_new[val_index]
        kfold_y_val = y_train_new[val_index]

        gc.collect()
        K.clear_session()

        model = get_model(features)

        ra_val = RocAucEvaluation(validation_data=([kfold_X_val,kfold_X_val_features], kfold_y_val), interval = 1)

        model.fit([kfold_X_train,kfold_X_features], kfold_y_train, batch_size=batch_size, epochs=epochs, verbose=1,
                callbacks = [ra_val])
        gc.collect()
        model.load_weights("best_weights.h5")

    predict = model.predict([x_test_new,features_test_new], batch_size=batch_size,verbose=1)
    score = roc_auc_score(y_test_new, predict)
    print("Test core:", score)
    break

print("Done")

(283759, 501)
